In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Install Kaggle, Clone Repo and Get Dataset

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
!git clone https://github.com/yusuffjamal3/Reviews_Query.git

Cloning into 'Reviews_Query'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 36 (delta 10), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [0]:
!cp Reviews_Query/kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

 95% 230M/242M [00:02<00:00, 108MB/s] 
100% 242M/242M [00:02<00:00, 116MB/s]


In [0]:
!unzip /content/amazon-fine-food-reviews.zip

Archive:  /content/amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              


In [1]:
# pd.set_option('display.max_colwidth', 999)

In [0]:
import pandas as pd
import numpy
import sqlite3
conn = sqlite3.connect("database.sqlite")

df = pd.read_sql_query("""select UserId, ProductId, 
                          ProfileName, Time,
                          Score, Text, COUNT(*)
                          FROM reviews
                          WHERE Score != 3
                          GROUP BY UserId
                          HAVING COUNT(*) > 1""", conn)
a = df.head()
df.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B007Y59HVM,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ET0,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B007Y59HVM,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ET0,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBE1U,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [0]:
df.sort_values('ProductId', inplace= True)
df.drop_duplicates(subset ="ProductId", 
                     keep = False, inplace = True)
df.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,"The same author wrote ""Where the Wild Things A...",2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,"It's just as described. Its minty, helps with ...",2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product is Awesome and Helpful for anyone...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much is the most cost effective wa...,2


In [0]:
positive_reviews = df['Score'].value_counts()[5] + df['Score'].value_counts()[4]
negative_reviews = df['Score'].value_counts()[2] + df['Score'].value_counts()[1]

print("The number of positive reviews are " + str(positive_reviews), 'reviews')
print("The number of negative reviews are " + str(negative_reviews), 'reviews')

The number of positive reviews are 9949 reviews
The number of negative reviews are 1844 reviews


In [0]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()
  
df['Text'] = df['Text'].apply(strip_tags)

df.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,"The same author wrote ""Where the Wild Things A...",2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,"It's just as described. Its minty, helps with ...",2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product is Awesome and Helpful for anyone...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much is the most cost effective wa...,2


In [0]:
import re

def remove_characters(text):
  return re.sub("[^A-z0-9\s]+", "", text)
  
df['Text'] = df['Text'].apply(remove_characters)

df.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,The same author wrote Where the Wild Things Ar...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,Its just as described Its minty helps with my ...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product is Awesome and Helpful for anyone...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much is the most cost effective wa...,2


In [0]:
def remove_non_english(text):
	return re.sub("[^a-zA-Z\s]", "", text)

df['Text'] = df['Text'].apply(remove_non_english)

df.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,The same author wrote Where the Wild Things Ar...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,Its just as described Its minty helps with my ...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product is Awesome and Helpful for anyone...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much is the most cost effective wa...,2


In [0]:
def check_length(text):
  text = text.split()
  contain = []
  
  for words in text:
    if len(words) > 2:
      contain.append(words)
      
  final = ' '.join(contain)
  return final
      
  
df['Text'] = df['Text'].apply(check_length)

df.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,The same author wrote Where the Wild Things Ar...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,Its just described Its minty helps with milddr...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product Awesome and Helpful for anyone wi...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much the most cost effective way a...,2


In [0]:
def lower_case(text):
  return text.lower()

df['Text'] = df['Text'].apply(lower_case)

df.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,the same author wrote where the wild things ar...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,its just described its minty helps with milddr...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,this product awesome and helpful for anyone wi...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,naturade total soy meal replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,buying this much the most cost effective way a...,2


In [0]:
import nltk

def remove_stopwords(text):
  try:
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    stop_words = set(stopwords.words( 'english' ))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    
  except:
    print("Downloading NLTK resources")
    nltk.download("stopwords")
    nltk.download("punkt")
    return remove_stopwords(text)
  
  return " ".join(filtered_sentence)

df['Text'] = df['Text'].apply(remove_stopwords)

df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,author wrote wild things carol king wrote grea...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,described minty helps milddry mouth provides r...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,product awesome helpful anyone asthma breathin...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,naturade total soy meal replacement powder adv...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,buying much cost effective way since lot makes...,2


In [0]:
# def stem_thy_words(text):
# 	from nltk.corpus import stopwords
# 	from nltk.stem import SnowballStemmer 
	
# 	snowball_stemmer = SnowballStemmer('english')
# 	word_tokens = nltk.word_tokenize(text)
# 	stemmed_word = [snowball_stemmer.stem(word) for word in word_tokens]
# 	return " ".join(stemmed_word)

# df['Text'] = df['Text'].apply(stem_thy_words)

# df.head()

KeyboardInterrupt: ignored